In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing.image import img_to_array,load_img
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, Dropout, Flatten, Dense, Conv2D, MaxPooling2D
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import classification_report,confusion_matrix
from tensorflow.keras.preprocessing import image 
from sklearn.model_selection import train_test_split

2021-09-29 12:19:51.246858: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0


In [3]:
train_genrator = ImageDataGenerator(rescale=1/255, 
                                   horizontal_flip=True,
                                   zoom_range=0.2,
                                    shear_range=0.2)
train_data = train_genrator.flow_from_directory("../input/real-life-industrial-dataset-of-casting-product/casting_data/casting_data/train",
                                               class_mode="binary",
                                               batch_size=32,
                                               target_size =(64,64),
                                               color_mode = "grayscale")

test_genrator = ImageDataGenerator(rescale=1/255)
test_data = train_genrator.flow_from_directory("../input/real-life-industrial-dataset-of-casting-product/casting_data/casting_data/test",
                                               class_mode="binary",
                                               batch_size=32,
                                               target_size =(64,64),
                                               color_mode = "grayscale")

Found 6633 images belonging to 2 classes.
Found 715 images belonging to 2 classes.


In [4]:
#Building the model 
#2 conv layers with kernel size of 3x3 followed bya maxpool with stride of 2 then 2 conv layers again 
#followed by maxpool of stride of 2 with pool size of 2x2 followd by 2 dense layers with activation 
# of relu and then a dense layer with 1 neuron with activation as sigmoid

Layers = [keras.layers.Conv2D(filters=16,kernel_size=(3,3),padding="same",input_shape=(64,64,1)),
         #Input is grayscale thus only one channel in input shape
          keras.layers.Activation("relu"),
          keras.layers.Conv2D(filters=16,kernel_size=(3,3),padding="same"),
          keras.layers.Activation("relu"),
          keras.layers.MaxPooling2D(pool_size=(2,2),strides=2),
          keras.layers.Conv2D(filters=16,kernel_size=(3,3),padding="same"),
          keras.layers.Activation("relu"),
          keras.layers.Conv2D(filters=16,kernel_size=(3,3),padding="same"),
          keras.layers.Activation("relu"),
          keras.layers.MaxPooling2D(pool_size=(2,2),strides=2),
          keras.layers.Flatten(),
          keras.layers.Dense(128),
          keras.layers.Activation("relu"),
          keras.layers.Dense(128),
          keras.layers.Activation("relu"),
          keras.layers.Dense(1,activation="sigmoid")]


            

In [5]:
# Adding layers to our sequential model
model = keras.models.Sequential(layers=Layers)

2021-09-29 12:19:59.410864: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2021-09-29 12:19:59.416834: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2021-09-29 12:19:59.467596: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:941] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-09-29 12:19:59.469648: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:00:04.0 name: Tesla P100-PCIE-16GB computeCapability: 6.0
coreClock: 1.3285GHz coreCount: 56 deviceMemorySize: 15.90GiB deviceMemoryBandwidth: 681.88GiB/s
2021-09-29 12:19:59.469860: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
2021-09-29 12:19:59.497385: I tensorflow/stream_executor/platform/def

In [6]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 64, 64, 16)        160       
_________________________________________________________________
activation (Activation)      (None, 64, 64, 16)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 64, 64, 16)        2320      
_________________________________________________________________
activation_1 (Activation)    (None, 64, 64, 16)        0         
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 32, 32, 16)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 32, 32, 16)        2320      
_________________________________________________________________
activation_2 (Activation)    (None, 32, 32, 16)        0

In [7]:
model.compile(optimizer="sgd",loss="binary_crossentropy",metrics=["accuracy"])

In [8]:
history = model.fit_generator(train_data,validation_data=test_data,epochs=20,verbose=2)

/opt/conda/lib/python3.7/site-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '
2021-09-29 12:20:01.849748: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)
2021-09-29 12:20:01.853242: I tensorflow/core/platform/profile_utils/cpu_utils.cc:112] CPU Frequency: 2000185000 Hz


Epoch 1/20


2021-09-29 12:20:02.485456: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublas.so.11
2021-09-29 12:20:03.339203: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublasLt.so.11
2021-09-29 12:20:03.364422: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudnn.so.8


208/208 - 72s - loss: 0.6866 - accuracy: 0.5640 - val_loss: 0.6685 - val_accuracy: 0.6336
Epoch 2/20
208/208 - 24s - loss: 0.6824 - accuracy: 0.5666 - val_loss: 0.6644 - val_accuracy: 0.6336
Epoch 3/20
208/208 - 21s - loss: 0.6745 - accuracy: 0.5747 - val_loss: 0.6436 - val_accuracy: 0.6336
Epoch 4/20
208/208 - 23s - loss: 0.6448 - accuracy: 0.6400 - val_loss: 0.6187 - val_accuracy: 0.6853
Epoch 5/20
208/208 - 25s - loss: 0.5991 - accuracy: 0.6808 - val_loss: 0.5557 - val_accuracy: 0.7273
Epoch 6/20
208/208 - 27s - loss: 0.5670 - accuracy: 0.7086 - val_loss: 0.5267 - val_accuracy: 0.7678
Epoch 7/20
208/208 - 25s - loss: 0.5455 - accuracy: 0.7226 - val_loss: 0.7966 - val_accuracy: 0.5371
Epoch 8/20
208/208 - 19s - loss: 0.5345 - accuracy: 0.7350 - val_loss: 0.5077 - val_accuracy: 0.7608
Epoch 9/20
208/208 - 20s - loss: 0.5279 - accuracy: 0.7380 - val_loss: 0.6057 - val_accuracy: 0.6727
Epoch 10/20
208/208 - 20s - loss: 0.5024 - accuracy: 0.7540 - val_loss: 0.4866 - val_accuracy: 0.7804


In [9]:
model.save("my_submission_model.h5")

In [10]:
def predictions(image_address):
    test_image = image.load_img(str(image_address),target_size=(64,64),color_mode= "grayscale")
    test_image = image.img_to_array(test_image)
    test_image = test_image/255
    test_image = np.expand_dims(test_image,axis=0)
    result = model.predict(test_image)
    if result[0]<=0.5:
        print("Defective")
    else:
        print("Not Defective ; ie:OK")

In [11]:

## Address from not defective 
address = "../input/real-life-industrial-dataset-of-casting-product/casting_data/casting_data/test/ok_front/cast_ok_0_1026.jpeg"
predictions(image_address=address)

Not Defective ; ie:OK


Predictions is correct

In [12]:
## Address from defective 

address = "../input/real-life-industrial-dataset-of-casting-product/casting_data/casting_data/test/def_front/cast_def_0_1189.jpeg"
predictions(address)

Defective


Prediction is correct